#Fertilizer Recommmendation System

In [71]:
# import the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')


In [4]:
Fr = pd.read_csv("Dataset/Fertilizer Prediction.csv")

In [6]:
Fr.shape

(99, 9)

In [8]:
Fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Temparature      99 non-null     int64 
 1   Humidity         99 non-null     int64 
 2   Moisture         99 non-null     int64 
 3   Soil Type        99 non-null     object
 4   Crop Type        99 non-null     object
 5   Nitrogen         99 non-null     int64 
 6   Potassium        99 non-null     int64 
 7   Phosphorous      99 non-null     int64 
 8   Fertilizer Name  99 non-null     object
dtypes: int64(6), object(3)
memory usage: 7.1+ KB


In [9]:
Fr.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,52,38,Sandy,Maize,37,0,0,Urea
1,29,52,45,Loamy,Sugarcane,12,0,36,DAP
2,34,65,62,Black,Cotton,7,9,30,14-35-14
3,32,62,34,Red,Tobacco,22,0,20,28-28
4,28,54,46,Clayey,Paddy,35,0,0,Urea


In [10]:
Fr.tail()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
94,25,50,32,Clayey,Pulses,24,0,19,28-28
95,30,60,27,Red,Tobacco,4,17,17,10-26-26
96,38,72,51,Loamy,Wheat,39,0,0,Urea
97,36,60,43,Sandy,Millets,15,0,41,DAP
98,29,58,57,Black,Sugarcane,12,0,10,20-20


Dataset Overview:
The dataset contains 99 entries and 9 columns.
The features include:
Environmental factors: Temperature, Humidity, Moisture.
Soil and Crop types: Categorical variables.
Soil Nutrients: Nitrogen, Potassium, Phosphorous.
Target Variable: Fertilizer Name (categorical).
Next, I'll check for missing values, analyze feature distributions, and then build a recommendation model.

In [14]:
missing_values = Fr.isnull().sum()
summary_stats = Fr.describe()
missing_values , summary_stats

(Temparature        0
 Humidity           0
 Moisture           0
 Soil Type          0
 Crop Type          0
 Nitrogen           0
 Potassium          0
 Phosphorous        0
 Fertilizer Name    0
 dtype: int64,
        Temparature  Humidity    Moisture   Nitrogen  Potassium  Phosphorous
 count    99.000000  99.000000  99.000000  99.000000  99.000000    99.000000
 mean     30.282828  59.151515  43.181818  18.909091   3.383838    18.606061
 std       3.502304   5.840331  11.271568  11.599693   5.814667    13.476978
 min      25.000000  50.000000  25.000000   4.000000   0.000000     0.000000
 25%      28.000000  54.000000  34.000000  10.000000   0.000000     9.000000
 50%      30.000000  60.000000  41.000000  13.000000   0.000000    19.000000
 75%      33.000000  64.000000  50.500000  24.000000   7.500000    30.000000
 max      38.000000  72.000000  65.000000  42.000000  19.000000    42.000000)

Data Analysis:
No missing values in the dataset.
Feature distribution:
Temperature ranges from 25°C to 38°C.
Humidity varies between 50% and 72%.
Moisture levels are between 25 and 65.
Nitrogen, Potassium, and Phosphorous vary widely, indicating diverse soil conditions.
Next, I'll analyze the distribution of the target variable (Fertilizer Name) and categorical features (Soil Type, Crop Type).

In [15]:
# Check unique values and distribution of categorical variables
fertilizer_counts = Fr["Fertilizer Name"].value_counts()
soil_counts = Fr["Soil Type"].value_counts()
crop_counts = Fr["Crop Type"].value_counts()

fertilizer_counts, soil_counts, crop_counts


(Fertilizer Name
 Urea        22
 DAP         18
 28-28       17
 14-35-14    14
 20-20       14
 17-17-17     7
 10-26-26     7
 Name: count, dtype: int64,
 Soil Type
 Loamy     21
 Sandy     20
 Clayey    20
 Black     19
 Red       19
 Name: count, dtype: int64,
 Crop Type
 Sugarcane      13
 Cotton         12
 Millets        11
 Paddy          10
 Pulses         10
 Wheat           9
 Tobacco         7
 Barley          7
 Oil seeds       7
 Ground Nuts     7
 Maize           6
 Name: count, dtype: int64)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical variables
label_encoders = {}
for col in ["Soil Type", "Crop Type", "Fertilizer Name"]:
    le = LabelEncoder()
    Fr[col] = le.fit_transform(Fr[col])
    label_encoders[col] = le

# Define features and target
X = Fr.drop(columns=["Fertilizer Name"])
y = Fr["Fertilizer Name"]

# Split into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check the shape of training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((79, 8), (20, 8), (79,), (20,))

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

accuracy


1.0

In [52]:
import pickle
with open("fertilizer_recommendation_model.pkl", "wb") as f:
    pickle.dump(final_model, f)


In [51]:
import joblib

# Load the saved model
model = joblib.load("fertilizer_recommendation_model.pkl")

# Load the label encoders for categorical variables
label_encoders = joblib.load("label_encoders.pkl")


In [42]:
import numpy as np

def predict_fertilizer(temperature, humidity, moisture, soil_type, crop_type, nitrogen, potassium, phosphorous):
    """
    Predicts the best fertilizer based on input conditions.
    """
    # Encode categorical inputs
    soil_encoded = label_encoders["Soil Type"].transform([soil_type])[0]
    crop_encoded = label_encoders["Crop Type"].transform([crop_type])[0]
    
    # Create input array
    input_data = np.array([[temperature, humidity, moisture, soil_encoded, crop_encoded, nitrogen, potassium, phosphorous]])
    
    # Predict fertilizer
    fertilizer_encoded = model.predict(input_data)[0]
    
    # Decode fertilizer prediction
    fertilizer_name = label_encoders["Fertilizer Name"].inverse_transform([fertilizer_encoded])[0]
    
    return fertilizer_name


In [66]:
# Example: Predict fertilizer for a Sugarcane crop in Loamy soil
fertilizer = predict_fertilizer(temperature=20, humidity=99, moisture=27, 
                                soil_type="Red", crop_type="Sugarcane", 
                                nitrogen=10, potassium=0, phosphorous=11)

print("Recommended Fertilizer:", fertilizer)


Recommended Fertilizer: 20-20


In [67]:
import joblib

# Save the trained model
joblib.dump(model, "fertilizer_recommendation_model.pkl")

# Save the label encoders for categorical data
joblib.dump(label_encoders, "label_encoders.pkl")

print("✅ Model and encoders saved successfully!")


✅ Model and encoders saved successfully!


In [74]:
#Scaling
# Scale the features using StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)